In [110]:
# Bibliotecas
from nltk.util import ngrams
import collections
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
# Carregar dados
proposicoes = pd.read_csv("../dados/proposicoes_normalizadas.csv")
proposicoes = list(proposicoes['texto'])

In [72]:
# Modelo de linguagem
nlp = spacy.load("pt_core_news_sm")

# N-Gramas

In [17]:
# Tokens coleção
tokens = []
for t in proposicoes:
    doc = nlp(t)
    tokens.extend([p.text for p in doc])

In [35]:
# N-gramas
N = 2
ngramas = ngrams(tokens, N)
frequencia_ngramas = collections.Counter(ngramas)
frequencia_ngramas.most_common(10)

[(('altera', 'lei'), 1429),
 (('dispõe', 'sobre'), 781),
 (('outras', 'providências'), 573),
 (('art', 'lei'), 545),
 (('constituição', 'federal'), 444),
 (('altera', 'art'), 402),
 (('dispor', 'sobre'), 376),
 (('acrescenta', 'art'), 223),
 (('art', 'constituição'), 209),
 (('código', 'penal'), 199)]

# Modelos Vetoriais

In [44]:
textos_exemplo = ['Este é o primeiro documento',
                  'Este documento é o segundo documento',
                  'E este é o terceiro, o derradeiro']

## Booleano

In [56]:
# Vetorizador booleano
vetorizador = CountVectorizer(binary=True)

In [57]:
matriz_booleana = vetorizador.fit_transform(textos_exemplo)


In [58]:
matriz_booleana_df = pd.DataFrame(matriz_booleana.todense(), index=['Doc1', 'Doc2', 'Doc3'],
                                  columns=vetorizador.get_feature_names())
matriz_booleana_df

,derradeiro,documento,este,primeiro,segundo,terceiro
Doc1,0,1,1,1,0,0
Doc2,0,1,1,0,1,0
Doc3,1,0,1,0,0,1


## TF

In [60]:
# Vetorizador TF
vetorizador = CountVectorizer()

In [61]:
matriz_tf = vetorizador.fit_transform(textos_exemplo)

In [62]:
matriz_tf_df = pd.DataFrame(matriz_tf.todense(), index=['Doc1', 'Doc2', 'Doc3'],
                            columns=vetorizador.get_feature_names())
matriz_tf_df

,derradeiro,documento,este,primeiro,segundo,terceiro
Doc1,0,1,1,1,0,0
Doc2,0,2,1,0,1,0
Doc3,1,0,1,0,0,1


## TF-IDF

In [64]:
# Vetorizador TF-IDF
vetorizador = TfidfVectorizer()

In [65]:
matriz_tfidf = vetorizador.fit_transform(textos_exemplo)

In [66]:
matriz_tfidf_df = pd.DataFrame(matriz_tfidf.todense(), index=['Doc1', 'Doc2', 'Doc3'],
                               columns=vetorizador.get_feature_names())
matriz_tfidf_df

,derradeiro,documento,este,primeiro,segundo,terceiro
Doc1,0.000000,0.547832,0.425441,0.720333,0.000000,0.000000
Doc2,0.000000,0.794803,0.308618,0.000000,0.522535,0.000000
Doc3,0.652491,0.000000,0.385372,0.000000,0.000000,0.652491


# Vetores de Palavras

In [75]:
proposicoes_tokenizadas = []
for t in proposicoes:
    proposicoes_tokenizadas.append(t.split())

In [91]:
model = Word2Vec(proposicoes_tokenizadas, size=100, window=5, min_count=1, workers=4)

In [94]:
model.wv.most_similar(positive=['criança'], topn=10)

[('adolescente', 0.9954029321670532),
 ('estatuto', 0.9944741725921631),
 ('dispõe', 0.9661090970039368),
 ('desarmamento', 0.9580464959144592),
 ('9.294', 0.9553644061088562),
 ('8.906', 0.952201247215271),
 ('idoso', 0.9511259198188782),
 ('incluir', 0.9477372169494629),
 ('1994', 0.9474542737007141),
 ('torcedor', 0.945695161819458)]

In [95]:
model.wv.most_similar(positive=['saúde'], topn=10)

[('serviços', 0.9991344213485718),
 ('sistema', 0.9988513588905334),
 ('público', 0.9985451698303223),
 ('correspondentes', 0.9982815384864807),
 ('públicos', 0.9981350302696228),
 ('social', 0.998095691204071),
 ('âmbito', 0.9974527359008789),
 ('recursos', 0.9973372220993042),
 ('pessoas', 0.9964507222175598),
 ('estrangeiros', 0.9962899684906006)]

In [109]:
model.wv.most_similar(positive=['valores'], topn=10)

[('qualquer', 0.9998366236686707),
 ('despesas', 0.9997918605804443),
 ('juros', 0.9997791051864624),
 ('regime', 0.9997761249542236),
 ('exterior', 0.9997750520706177),
 ('pagamento', 0.9997730255126953),
 ('ações', 0.9997718334197998),
 ('comunicação', 0.9997693300247192),
 ('internet', 0.9997621178627014),
 ('dados', 0.9997572898864746)]

# Similaridade textual

In [111]:
similaridades = cosine_similarity(matriz_tfidf)

In [113]:
similaridades_df = pd.DataFrame(similaridades, columns=['Doc1', 'Doc2', 'Doc3'],
                                index=['Doc1', 'Doc2', 'Doc3'])
similaridades_df

,Doc1,Doc2,Doc3
Doc1,1.000000,0.566717,0.163953
Doc2,0.566717,1.000000,0.118933
Doc3,0.163953,0.118933,1.000000


In [114]:
textos_exemplo

['Este é o primeiro documento',
 'Este documento é o segundo documento',
 'E este é o terceiro, o derradeiro']